# Species Analysis

## import

In [2]:
import os

In [3]:
import pandas as pd

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
import json

In [7]:
%run ../../Util/FileFolder

created


In [8]:
%run ../../Util/Variable

# subfolders

In [9]:
subfolder={}

## sub folder count

In [10]:
subfolderCount=num_sub_folders(original_dataset_dir)
subfolder['count']=subfolderCount
subfolder

{'count': 7}

### Subfolder names


In [11]:
folders_names=sub_folders(original_dataset_dir)

In [12]:
folders=list(
    map(
        lambda name: 
            {'name':name,
             'path':os.path.join(original_dataset_dir,name),
             'filecount':count_files_in_folder(os.path.join(original_dataset_dir,name)),
             'volume':{
                 'GB':folder_size_in_gb(os.path.join(original_dataset_dir,name)),
                 "KB":folder_size_in_kb(os.path.join(original_dataset_dir,name)),
                "MB":folder_size_in_mb(os.path.join(original_dataset_dir,name))
             }
            },folders_names))
folders

[{'name': '.git',
  'path': 'D:\\\\Research\\\\ayush-vision\\.git',
  'filecount': 318,
  'volume': {'GB': 0.0005262289196252823,
   'KB': 551.791015625,
   'MB': 0.5388584136962891}},
 {'name': 'Algorithums',
  'path': 'D:\\\\Research\\\\ayush-vision\\Algorithums',
  'filecount': 1,
  'volume': {'GB': 1.0328367352485657e-06,
   'KB': 1.0830078125,
   'MB': 0.0010576248168945312}},
 {'name': 'config',
  'path': 'D:\\\\Research\\\\ayush-vision\\config',
  'filecount': 2,
  'volume': {'GB': 2.493150532245636e-06,
   'KB': 2.6142578125,
   'MB': 0.0025529861450195312}},
 {'name': 'DataSet',
  'path': 'D:\\\\Research\\\\ayush-vision\\DataSet',
  'filecount': 5,
  'volume': {'GB': 1.9213184714317322e-05,
   'KB': 20.146484375,
   'MB': 0.019674301147460938}},
 {'name': 'Summary',
  'path': 'D:\\\\Research\\\\ayush-vision\\Summary',
  'filecount': 1,
  'volume': {'GB': 5.4016709327697754e-08,
   'KB': 0.056640625,
   'MB': 5.53131103515625e-05}},
 {'name': 'Testing',
  'path': 'D:\\\\Researc

### subfolder details

In [13]:
subfolders=[]

In [14]:
for subfolder in folders:
    subfoldernames=sub_folders(subfolder['path'])
    subchild=list(
    map(
        lambda name: 
            {'name':name,
             'path':os.path.join(subfolder['path'],name),
             'filecount':count_files_in_folder(os.path.join(subfolder['path'],name)),
             'volume':{
                 'GB':folder_size_in_gb(os.path.join(subfolder['path'],name)),
                 "KB":folder_size_in_kb(os.path.join(subfolder['path'],name)),
                "MB":folder_size_in_mb(os.path.join(subfolder['path'],name))
             }
            },subfoldernames))
    subfolder['subfolders']=subchild
    subfolders.append(subfolder)
subfolders
    

[{'name': '.git',
  'path': 'D:\\\\Research\\\\ayush-vision\\.git',
  'filecount': 318,
  'volume': {'GB': 0.0005262289196252823,
   'KB': 551.791015625,
   'MB': 0.5388584136962891},
  'subfolders': [{'name': 'fsmonitor--daemon',
    'path': 'D:\\\\Research\\\\ayush-vision\\.git\\fsmonitor--daemon',
    'filecount': 0,
    'volume': {'GB': 0.0, 'KB': 0.0, 'MB': 0.0}},
   {'name': 'hooks',
    'path': 'D:\\\\Research\\\\ayush-vision\\.git\\hooks',
    'filecount': 14,
    'volume': {'GB': 2.4053268134593964e-05,
     'KB': 25.2216796875,
     'MB': 0.02463054656982422}},
   {'name': 'info',
    'path': 'D:\\\\Research\\\\ayush-vision\\.git\\info',
    'filecount': 1,
    'volume': {'GB': 2.2351741790771484e-07,
     'KB': 0.234375,
     'MB': 0.0002288818359375}},
   {'name': 'logs',
    'path': 'D:\\\\Research\\\\ayush-vision\\.git\\logs',
    'filecount': 24,
    'volume': {'GB': 4.45488840341568e-05,
     'KB': 46.712890625,
     'MB': 0.04561805725097656}},
   {'name': 'objects',
 

In [15]:
today_collection_path=os.path.join(today_summary_path,'collection')
reportjsonfile=os.path.join(today_collection_path,'reportjsonfile.json')

In [16]:
with open(reportjsonfile, 'r+') as f:
        # Load existing data from JSON
        existing_data = json.load(f)
        
        # Append new data to existing JSON data
        existing_data['subfolder']=subfolders
        
        # Write updated data back to JSON file
        f.seek(0)
        json.dump(existing_data, f, indent=4)
        f.truncate()

In [31]:
subfolderdetails =list(map(lambda folder: {
    'name':folder['name'],
    'path':folder['path'],
    'files':folder['filecount'],
    'volume(MB)':folder['volume']['MB'],
    },subfolders)) 
subfolderdetails_li=list(map(lambda folder: [
    folder['name'],
    folder['path'],
    folder['filecount'],
    folder['volume']['MB'],
    ],subfolders)) 

In [32]:
save_to_csv_filedetails(data=subfolderdetails_li,colums=['Names','Path','Files',"Volume MB"],csv_filename='reportcsv.csv',save_folder=today_collection_path)

7